<a href="https://colab.research.google.com/github/leejunho12316/HonGongMachine/blob/main/%EC%95%99%EC%83%81%EB%B8%94_%EB%A6%AC%ED%8A%B8%EB%A6%AC%EB%B2%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install langchain openai tiktoken langchain-community rank_bm25 sentence-transformers chromadb pypdf langchain-huggingface langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.0/331.0 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 105.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 91.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

## pdf 파일로드 하고 쪼개기
loader = PyPDFLoader('https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf')
pages = loader.load_and_split()
print(len(pages))

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
texts = text_splitter.split_documents(pages)
print(len(texts))

27
69


#BM25

In [ ]:
from langchain_community.retrievers import BM25Retriever
bm25_retriever = BM25Retriever.from_documents(texts, k=2)

results = bm25_retriever.invoke('삼성전자의 인재상은?')
for r in results:
  print(r)
  print()

#VectorDB - Chroma

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
model_huggingface = HuggingFaceEmbeddings(model_name='BAAI/bge-m3')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [6]:
from langchain_community.vectorstores import Chroma
chroma_db = Chroma.from_documents(texts, model_huggingface)
chroma_retriever = chroma_db.as_retriever(search_kwargs={'k' : 2})

In [ ]:
results = chroma_retriever.invoke('삼성전자의 인재상은?')
for r in results:
  print(r)
  print()

#Ensemble Retriever

In [9]:
from typing import List, Optional
from langchain_core.documents import Document

class EnsembleRetriever:
    """
    BM25 + Dense retriever 앙상블.
    Reciprocal Rank Fusion (RRF) 알고리즘으로 점수 결합.
    """
    def __init__(self, retrievers: List, weights: Optional[List[float]] = None, c: int = 60, id_key: Optional[str] = None):
        assert len(retrievers) == len(weights), "retrievers와 weights 길이가 다릅니다."
        self.retrievers = retrievers
        self.weights = weights
        self.c = c
        self.id_key = id_key

    def _get_doc_id(self, doc: Document) -> str:
        """문서 중복 판단용 ID 추출"""
        if self.id_key and self.id_key in doc.metadata:
            return str(doc.metadata[self.id_key])
        return doc.page_content[:150]

    def _fuse_results(self, all_results: List[List[Document]]) -> List[Document]:
        """Reciprocal Rank Fusion 계산"""
        scores: Dict[str, Dict[str, Any]] = {}

        for i, docs in enumerate(all_results):
            weight = self.weights[i] if self.weights else 1.0

            for rank, doc in enumerate(docs):
                doc_id = self._get_doc_id(doc)

                score = weight / (rank + 1 + self.c)

                if doc_id not in scores:
                    scores[doc_id] = {"doc": doc, "score": score}
                else:
                    scores[doc_id]["score"] += score

        sorted_docs = sorted(scores.values(), key=lambda x: x["score"], reverse=True)
        return [item["doc"] for item in sorted_docs]

    def invoke(self, query: str, k: int = 4) -> List[Document]:
        """모든 retriever에서 검색 후 점수 융합"""
        all_results = []
        for retriever in self.retrievers:
            if hasattr(retriever, "get_relevant_documents"):
                docs = retriever.get_relevant_documents(query)
            elif hasattr(retriever, "invoke"):
                docs = retriever.invoke(query)
            else:
                raise AttributeError(f"{retriever} does not support retrieval.")
            all_results.append(docs[:k])

        fused_docs = self._fuse_results(all_results)
        return fused_docs[:k]

In [18]:
ensemble_retriever = EnsembleRetriever(
    retrievers = [bm25_retriever, chroma_retriever],
    weights = [0.3,0.7]
)
results = ensemble_retriever.invoke('삼성전자의 인재상은?')
for r in results:
  print(r)
  print()

page_content='2
01 삼성전자 기업분석
(Samsung Electronics Co., Ltd)
Ⅰ 기업 일반 
1  기업개요
1) 기업소개 
본사주소 경기도 수원시 영통구 삼성로 129(매탄동 416)
사업분야 삼성그룹의 대표 기업으로 휴대폰, 정보통신기기, 반도체, TV 등을 생산 판매하는 제조업체
홈페이지 www.samsung.com/sec 구분 전기전자 대기업  
설립일 1961년 07월 01일 대표이사 권오현 
총자산1) 244조 매출액2) 200조
임직원수 95,374명 
∙ 1975년 1월 주식시장 상장
∙ 1984년 2월 삼성전자공업주식회사->삼성전자주식회사로 사명 변경 
∙ CE(Consumer Electronics), IM(Information technology & Mobile communications), DS(Device Solutions) 
3개의 부문으로 나누어 독립 경영.
부문 제품
CE TV, 모니터, 냉장고, 세탁기, 에어컨, 프린터, 의료기기 등' metadata={'page_label': '1', 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'page': 0, 'creator': 'nPDF (pdftk 1.41)', 'creationdate': '2017-08-16T00:21:02-08:00', 'total_pages': 27, 'moddate': '2017-08-16T00:21:02-08:00', 'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)'}

page_content='2015년말 기준 삼성전자의 총자산 금액은 약 169조에 달하며, 부채는 32.5조, 자본총계는 136조에 이르고 
있다.' metadata={'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (

#Hybrid Search

In [19]:
from langchain_core.documents import Document
from typing import List, Set

# 두 검색 방법을 별도로 실행하고 결과 통합
def hybrid_search(query: str, k: int = 5) -> List[Document]:
    """
    BM25와 임베딩 기반 검색을 결합한 하이브리드 검색 구현

    이 함수는 키워드 기반 검색(BM25)과 의미 기반 검색(임베딩)을 모두 수행하고
    결과를 번갈아가며 통합하여 다양한 관점의 검색 결과를 제공합니다.

    Args:
        query (str): 검색 쿼리
        k (int): 반환할 최대 문서 수

    Returns:
        List[Document]: 중복이 제거된 통합 검색 결과
    """
    # BM25 검색 결과 가져오기 (키워드 기반 검색)
    bm25_results = bm25_retriever.invoke(query)

    # 임베딩 검색 결과 가져오기 (의미 기반 검색)
    embedding_results = chroma_retriever.invoke(query)

    # 결과 통합 및 중복 제거를 위한 변수 초기화
    seen_contents: Set[str] = set()  # 이미 처리된 문서 내용을 추적
    hybrid_results: List[Document] = []  # 최종 결과를 저장할 리스트

    # 두 결과 세트를 번갈아가며 통합 (라운드 로빈 방식)
    for i in range(max(len(bm25_results), len(embedding_results))):
        # BM25 결과 처리 (있는 경우)
        if i < len(bm25_results):
            doc = bm25_results[i]
            # 중복 문서 확인 및 추가
            if doc.page_content not in seen_contents:
                seen_contents.add(doc.page_content)
                hybrid_results.append(doc)

        # 임베딩 검색 결과 처리 (있는 경우)
        if i < len(embedding_results):
            doc = embedding_results[i]
            # 중복 문서 확인 및 추가
            if doc.page_content not in seen_contents:
                seen_contents.add(doc.page_content)
                hybrid_results.append(doc)

        # 요청한 수(k)만큼 문서를 찾았으면 중단
        if len(hybrid_results) >= k:
            break

    # 최대 k개의 결과 반환
    return hybrid_results[:k]

In [21]:
results = hybrid_search('삼성전자의 사장님은?', k=5)

In [22]:
for r in results:
  print(r)
  print()

page_content='2015년말 기준 삼성전자의 총자산 금액은 약 169조에 달하며, 부채는 32.5조, 자본총계는 136조에 이르고 
있다.' metadata={'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (pdftk 1.41)', 'creationdate': '2017-08-16T00:21:02-08:00', 'moddate': '2017-08-16T00:21:02-08:00', 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'total_pages': 27, 'page': 14, 'page_label': '15'}

page_content='2
01 삼성전자 기업분석
(Samsung Electronics Co., Ltd)
Ⅰ 기업 일반 
1  기업개요
1) 기업소개 
본사주소 경기도 수원시 영통구 삼성로 129(매탄동 416)
사업분야 삼성그룹의 대표 기업으로 휴대폰, 정보통신기기, 반도체, TV 등을 생산 판매하는 제조업체
홈페이지 www.samsung.com/sec 구분 전기전자 대기업  
설립일 1961년 07월 01일 대표이사 권오현 
총자산1) 244조 매출액2) 200조
임직원수 95,374명 
∙ 1975년 1월 주식시장 상장
∙ 1984년 2월 삼성전자공업주식회사->삼성전자주식회사로 사명 변경 
∙ CE(Consumer Electronics), IM(Information technology & Mobile communications), DS(Device Solutions) 
3개의 부문으로 나누어 독립 경영.
부문 제품
CE TV, 모니터, 냉장고, 세탁기, 에어컨, 프린터, 의료기기 등' metadata={'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'moddate': '2017

#RAG + OpenAI API

In [28]:
import os

os.environ['OPENAI_API_KEY'] =  ""

In [29]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-4.1", temperature=0)

prompt_template = ChatPromptTemplate.from_template("""
아래의 문서를 참고하여 질문에 대한 답변을 작성하세요.

질문: {question}

참고 문서:
{context}

답변:
""")

In [30]:
def ask_question(question : str, retriever):
  #문서 RAG
  rag_docs = retriever.invoke(question)
  context = "\n\n".join(doc.page_content for doc in rag_docs)

  prompt = prompt_template.format_messages(question=question, context=context)

  result = llm.invoke(prompt)

  return result.content

In [34]:
ask_question('삼성전자에 가장 빠르게 들어갈 수 있는 방법은?', ensemble_retriever)

'삼성전자에 가장 빠르게 들어갈 수 있는 방법은 다음과 같습니다.\n\n1. **채용 절차를 정확히 이해하고 준비하기**\n   - 삼성전자의 채용 절차는 1차 서류전형 → 2차 직무적성검사 → 3차 면접 순으로 진행됩니다. 각 단계별로 요구하는 사항을 미리 파악하고 준비하는 것이 중요합니다.\n\n2. **지원 자격을 미리 갖추기**\n   - 병역필 또는 면제자, 해외여행에 결격사유가 없어야 하며, 영어회화 자격(예: 오픽, 토익스피킹 등)을 보유해야 합니다. 미리 영어회화 자격증을 취득해 두면 지원 시 시간을 단축할 수 있습니다.\n\n3. **서류전형 준비**\n   - 성적, 출결, 생활태도, 지원동기 및 포부, 보유자격 등 다양한 요소가 평가되므로, 학점 관리와 자기소개서, 자격증 준비를 미리 해두는 것이 좋습니다.\n\n4. **직무적성검사 대비**\n   - 삼성의 직무적성검사(GSAT)는 기초능력과 직무능력을 평가합니다. 시중에 나와 있는 기출문제집이나 온라인 강의를 활용해 미리 연습하면 빠르게 합격할 수 있습니다.\n\n5. **면접 준비**\n   - 임원면접과, 엔지니어/SW직의 경우 기술면접이 추가될 수 있으므로, 지원 분야에 맞는 면접 준비를 철저히 해야 합니다.\n\n6. **채용공고 상시 확인 및 빠른 지원**\n   - 삼성전자는 정기공채 외에도 수시채용을 진행하므로, 채용공고를 자주 확인하고, 공고가 뜨는 즉시 지원하면 빠르게 입사 절차를 밟을 수 있습니다.\n\n**요약:**  \n삼성전자에 가장 빠르게 들어가려면, 지원 자격(특히 영어회화 자격증 등)을 미리 갖추고, 서류·적성검사·면접을 철저히 준비한 뒤, 채용공고를 상시 확인하여 바로 지원하는 것이 가장 효과적입니다. 또한, 자신의 전공과 경력, 자격증이 삼성전자가 필요로 하는 분야와 잘 맞는지 미리 점검하는 것도 중요합니다.'

In [35]:
ask_question('삼성전자에 가장 빠르게 들어갈 수 있는 방법은?', bm25_retriever)

'삼성전자에 가장 빠르게 들어갈 수 있는 방법은 다음과 같습니다.\n\n삼성전자는 V낸드, 낸드플래시 등 반도체 분야에서 세계적인 경쟁력을 가지고 있으며, 특히 V낸드와 같은 첨단 기술을 빠르게 개발·생산하고 있습니다. 따라서 삼성전자에 빠르게 입사하기 위해서는 다음과 같은 방법이 효과적입니다.\n\n1. **반도체 및 관련 전공 역량 강화**  \n삼성전자는 V낸드, 낸드플래시 등 반도체 기술에 강점을 가지고 있으므로, 전자공학, 재료공학, 물리학 등 관련 전공을 선택하고, 반도체 공정, 메모리 설계, 비메모리 반도체(AP 등) 등 실무에 필요한 지식을 갖추는 것이 중요합니다.\n\n2. **인턴십 및 산학협력 프로그램 활용**  \n삼성전자는 인턴십, 산학협력, 채용연계형 인턴 등 다양한 경로로 인재를 선발합니다. 대학 재학 중 삼성전자 인턴십에 지원하거나, 산학협력 프로그램에 참여하면 정규직 전환의 기회를 빠르게 얻을 수 있습니다.\n\n3. **공채 및 수시채용 적극 지원**  \n삼성전자는 정기공채(삼성그룹 공채)와 수시채용을 병행하고 있습니다. 채용 일정에 맞춰 빠르게 지원하고, 삼성직무적성검사(GSAT) 등 전형 준비를 철저히 하면 빠른 입사가 가능합니다.\n\n4. **핵심 기술 및 트렌드 파악**  \n삼성전자가 집중하는 V낸드, 낸드플래시, AP 등 핵심 기술과 스마트워크 등 최신 업무 트렌드를 파악하고, 관련 프로젝트나 논문, 자격증 등을 준비하면 경쟁력을 높일 수 있습니다.\n\n요약하면, 삼성전자에 가장 빠르게 들어가려면 반도체 등 핵심 분야의 역량을 갖추고, 인턴십·산학협력 등 채용 연계 프로그램을 적극 활용하며, 공채 및 수시채용에 신속하게 지원하는 것이 효과적입니다.'

In [37]:
ask_question('삼성전자에 가장 빠르게 들어갈 수 있는 방법은?', chroma_retriever)

'삼성전자에 가장 빠르게 들어갈 수 있는 방법은 공식 채용 절차를 신속하게 준비하고, 각 전형 단계에서 요구하는 조건을 미리 갖추는 것입니다. 삼성전자의 채용 절차는 다음과 같습니다.\n\n1. **서류전형**:  \n   - 성적, 출결, 생활태도, 지원동기 및 포부, 보유자격 등을 종합적으로 평가합니다.  \n   - 따라서, 학점 관리, 성실한 학교생활, 명확한 지원동기와 포부를 미리 준비하고, 관련 자격증(특히 영어회화 자격)을 취득해 두는 것이 중요합니다.\n\n2. **직무적성검사**:  \n   - 기초능력과 직무능력을 평가하는 시험이 있으므로, 삼성 직무적성검사(GSAT) 기출문제와 유사문제를 미리 충분히 연습해야 합니다.\n\n3. **면접**:  \n   - 임원면접이 기본이며, 엔지니어직 및 S/W직은 별도의 기술면접이 있을 수 있습니다.  \n   - 지원 분야에 맞는 전공지식과 실무 역량, 그리고 회사와 직무에 대한 이해도를 높여 면접을 준비해야 합니다.\n\n**가장 빠르게 입사하는 방법 요약**\n- 채용공고를 수시로 확인하고, 지원자격(병역필 또는 면제, 영어회화자격 등)을 미리 갖추세요.\n- 서류전형에 필요한 성적, 자격증, 자기소개서 등을 미리 준비하세요.\n- 직무적성검사(GSAT) 대비를 충분히 하세요.\n- 면접 준비(임원/기술)를 철저히 하세요.\n\n즉, 삼성전자에 가장 빠르게 들어가려면, 채용공고가 뜨자마자 지원할 수 있도록 모든 준비를 사전에 마치고, 각 전형별로 요구하는 조건을 미리 갖추는 것이 가장 효과적인 방법입니다.'